In [5]:
from pyspark import SparkContext
from spark_sklearn import GridSearchCV
from pylab import *

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, make_scorer
from sklearn.preprocessing import MinMaxScaler 
from time import time
from math import sqrt
from sklearn.linear_model import ElasticNet, SGDRegressor, LinearRegression, ElasticNetCV, LassoCV, Lasso, Ridge, RidgeCV, BayesianRidge, ARDRegression
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor

from matplotlib import pyplot
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
%matplotlib inline


In [7]:
data = pd.read_csv('/Users/Andy/Desktop/Bitcoin/Data/Bitcoin_raw.csv')
print('n_features:', len(data.iloc[0]))
print('n_samples:', len(data))
data.head()

n_features: 15
n_samples: 2007


,date,open,high,low,close,volume,marketcap,BCHAIN-DIFF,BCHAIN-AVBLS,BCHAIN-MIREV,BCHAIN-CPTRA,BCHAIN-NTRAN,BCHAIN-HRATE,BCHAIN-CPT,BCHAIN-NTRBL
0,30/6/19,11931.99,12178.38,10799.01,10817.16,27256473494,1.924420e+11,7.930000e+12,1.148651,27146866.15,76.109437,356682,68631991.64,76.109437,2049.896552
1,29/6/19,12400.76,12400.91,11508.38,11959.37,29923961128,2.127400e+11,7.930000e+12,1.255488,25894830.35,63.608971,407094,64293187.57,63.608971,2497.509202
2,28/6/19,11162.17,12445.18,10914.49,12407.33,35087757766,2.206810e+11,7.870000e+12,1.233318,23373593.45,64.737830,361050,56010016.17,64.737830,2542.605634
3,27/6/19,13017.12,13311.14,10491.85,11182.81,39977475223,1.988790e+11,7.410000e+12,1.240849,28500796.50,71.319031,399624,60773333.69,71.319031,2421.963636
4,26/6/19,11778.58,13796.49,11755.60,13016.23,45105733173,2.314620e+11,7.410000e+12,1.245764,22019741.67,60.931532,361385,53775192.23,60.931532,2475.239726


## Data Period Selection

In [23]:
data[181:546]

,date,open,high,low,close,volume,marketcap,BCHAIN-DIFF,BCHAIN-AVBLS,BCHAIN-MIREV,BCHAIN-CPTRA,BCHAIN-NTRAN,BCHAIN-HRATE,BCHAIN-CPT,BCHAIN-NTRBL
181,31/12/18,3866.84,3868.74,3725.87,3742.70,4661840806,6.533150e+10,5.110000e+12,0.762487,8.485748e+06,31.705946,267639,44676132.24,31.705946,1520.676136
182,30/12/18,3822.38,3901.91,3797.22,3865.95,4770578575,6.747551e+10,5.110000e+12,0.973710,6.808600e+06,25.694769,264980,35030149.14,25.694769,1920.144928
183,29/12/18,3932.49,3963.76,3820.41,3820.41,4991655917,6.667224e+10,5.110000e+12,0.930544,7.882682e+06,25.570957,308267,42391557.29,25.570957,1845.910180
184,28/12/18,3653.13,3956.14,3642.63,3923.92,5631554348,6.847184e+10,5.110000e+12,0.885621,8.214951e+06,27.381344,300020,44168448.91,27.381344,1724.252874
185,27/12/18,3854.69,3874.42,3645.45,3654.83,5130222366,6.376876e+10,5.110000e+12,0.889581,7.569380e+06,26.908087,281305,39853140.69,26.908087,1791.751592
186,26/12/18,3819.67,3893.36,3769.86,3857.30,5326547918,6.729282e+10,5.110000e+12,0.821911,7.649830e+06,28.861619,265052,40360824.01,28.861619,1666.993711
187,25/12/18,4081.03,4089.56,3760.02,3815.49,6158207293,6.655603e+10,5.110000e+12,0.932735,8.035502e+06,29.510064,272297,38583932.39,29.510064,1791.427632
188,24/12/18,4000.33,4271.79,4000.33,4078.60,7240968501,7.113755e+10,5.110000e+12,0.740468,8.922675e+06,35.585509,250739,44676132.24,35.585509,1424.653409
189,23/12/18,4020.99,4085.72,3976.41,3998.98,6151275490,6.974122e+10,5.110000e+12,0.806539,8.628688e+06,34.347687,251216,44422290.57,34.347687,1435.520000
190,22/12/18,3898.08,4014.18,3855.74,4014.18,5605823233,6.999751e+10,5.110000e+12,1.048346,7.151635e+06,25.488030,280588,35537832.46,25.488030,2004.200000


In [24]:

# Data period
periods = [2014,2015,2016,2017,2018,2019]
period_indices = [1642,1277,911,546,181,0]
#periods = [2019,2018,2017,2016,2015,2014]
#period_indices = [2189,1823,1458,1093,728,362,0] #[2610,2540,2470,2400,2330,2260,2189]

# Training with different data period
period_rmse = []
for index in period_indices:
    train_data = data.iloc[index:,:].values
    n_samples = len(train_data)
    Y = train_data[1:,0].reshape(-1,1)
    X = train_data[0:n_samples-1,1:]
    rmse = train_model(X,Y,Lasso(),n_validation=91,predict=False,return_y=False)
    period_rmse.append(rmse)    
# Compare test RMSE of different period
print('RMSE of different periods')
for i in range(0,len(periods)):
    print(str(periods[i])+': ',str(period_rmse[i]))
print('Best period:', str(periods[np.argmin(period_rmse)]))

# Prepare training data for all models
start = period_indices[np.argmin(period_rmse)] # use best period

train_data = data.iloc[start:,:].values
n_samples = len(train_data)
Y = train_data[1:,0].reshape(-1,1)
X = train_data[0:n_samples-1,1:]


NameError: name 'train_model' is not defined